
**Import the necessary librairies**

In [ ]:
%%time
import numpy as np
import pandas as pd 
import os
import sys
import cv2
import plotly.express as px
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
import pickle
import tensorflow as tf
from sklearn.model_selection import train_test_split
from matplotlib import pyplot
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import tensorflow.keras as keras
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dropout, BatchNormalization, LeakyReLU, Activation
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from multiprocessing import Queue
from keras.utils import to_categorical
from tensorflow.keras.losses import categorical_crossentropy
from keras.utils import plot_model
import seaborn as sns
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
from IPython.display import HTML
from keras.applications import ResNet50

**Import Fer2013 dataset and Display Class Distribution**

In [ ]:
root_path = '/kaggle/input/fer_2013/fer/'
class_names = sorted(os.listdir(root_path))
n_classes = len(class_names)

# Class Distribution
class_dis = [len(os.listdir(root_path + name)) for name in class_names]

# Show
print(f"Total Number of Classes : {n_classes} \nClass Names : {class_names}")

**Pre-processing step**

In [ ]:
DATADIR = '/kaggle/input/fer_2013/fer/'

CATEGORIES = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sadness', 'surprise']

for category in CATEGORIES:  
    path = os.path.join(DATADIR,category)  
    for img in os.listdir(path):  
        img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)  # convert to array
        plt.imshow(img_array, cmap='gray')  # graph it
        plt.show()  # display!
        print(img_array)
        print(img_array.shape)
        break  
    break 

**Creating training Data**

In [ ]:
IMG_SIZE = 48
training_data = []

def create_training_data():
    for category in CATEGORIES:  

        path = os.path.join(DATADIR,category)  
        class_num = CATEGORIES.index(category)  

        for img in tqdm(os.listdir(path)):  
            try:
                img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)  # convert to array
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize to normalize data size
                training_data.append([new_array, class_num])  # add this to the training data
            except Exception as e:  
                pass
            #except OSError as e:
            #    print("OSErrroBad img most likely", e, os.path.join(path,img))
            #except Exception as e:
            #    print("general exception", e, os.path.join(path,img))

create_training_data()

print(len(training_data))

In [ ]:
random.shuffle(training_data)
A2= []
B2= []
for features,label in training_data:
    A2.append(features)
    B2.append(label)

print(A2[0].reshape(-1, IMG_SIZE, IMG_SIZE, 1))

A2= np.array(A2).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

pickle_out = open("A2.pickle","wb")
pickle.dump(A2, pickle_out)
pickle_out.close()

pickle_out = open("B2.pickle","wb")
pickle.dump(B2, pickle_out)
pickle_out.close()

In [ ]:
pickle_in = open("A2.pickle","rb")
A2 = pickle.load(pickle_in)
pickle_in = open("B2.pickle","rb")
B2= pickle.load(pickle_in)
B2 = to_categorical(B2, num_classes=7)
x_train,x_valid,y_train,y_valid=train_test_split(A2,B2,test_size=0.2)
x_train = x_train / 255.
x_valid = x_valid / 255.
num_classes=y_train.shape[1]

**Building the deeep convolutional neural network**

In [ ]:

def build_network(optim):
   

    network = Sequential(name='Deep_CNN')

    network.add(
        Conv2D(
            filters=64,
            kernel_size=(5,5),
            input_shape=(48, 48, 1),
            activation='elu',
            padding='same',
            kernel_initializer='he_normal'
        )
    )
    network.add(BatchNormalization(name='batchnormalization_1'))
    network.add(
        Conv2D(
            filters=64,
            kernel_size=(5,5),
            activation='elu',
            padding='same',
            kernel_initializer='he_normal'
        )
    )
    network.add(BatchNormalization(name='batchnormalization_2'))
    
    network.add(MaxPooling2D(pool_size=(2,2), name='maxpooling2d_1'))
    network.add(Dropout(0.4, name='dropout_1'))

    network.add(
        Conv2D(
            filters=128,
            kernel_size=(3,3),
            activation='elu',
            padding='same',
            kernel_initializer='he_normal'
        )
    )
    network.add(BatchNormalization(name='batchnormalization_3'))
    network.add(
        Conv2D(
            filters=128,
            kernel_size=(3,3),
            activation='elu',
            padding='same',
            kernel_initializer='he_normal'
        )
    )
    network.add(BatchNormalization(name='batchnormalization_4'))
    
    network.add(MaxPooling2D(pool_size=(2,2), name='maxpooling2d_2'))
    network.add(Dropout(0.4, name='dropout_2'))

    network.add(
        Conv2D(
            filters=256,
            kernel_size=(3,3),
            activation='elu',
            padding='same',
            kernel_initializer='he_normal'
        )
    )
    network.add(BatchNormalization(name='batchnormalization_5'))
    network.add(
        Conv2D(
            filters=256,
            kernel_size=(3,3),
            activation='elu',
            padding='same',
            kernel_initializer='he_normal'
        )
    )
    network.add(BatchNormalization(name='batchnormalization_6'))
    
    network.add(MaxPooling2D(pool_size=(2,2), name='maxpooling2d_3'))
    network.add(Dropout(0.5, name='dropout_3'))

    network.add(Flatten(name='flatten'))
        
    network.add(
        Dense(
            128,
            activation='elu',
            kernel_initializer='he_normal'
        )
    )
    network.add(BatchNormalization(name='batchnormalization_7'))
    
    network.add(Dropout(0.6, name='dropout_4'))
    
    network.add(
        Dense(
            num_classes,
            activation='softmax',
            name='out_layer'
        )
    )
    METRICS = [
        tf.keras.metrics.BinaryAccuracy(name='accuracy'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.AUC(name='auc')
    ]
    network.compile(
        loss='categorical_crossentropy',
        optim=optim,
        metrics= METRICS 
    )
    
    network.summary()
    
    return network

early_stopping = EarlyStopping(
    monitor='val_accuracy',
    min_delta=0.00005,
    patience=11,
    verbose=1,
    restore_best_weights=True,
)

lr_scheduler = ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.5,
    patience=7,
    min_lr=1e-7,
    verbose=1,
)

callbacks = [
    early_stopping,
    lr_scheduler,
]
batch_size = 32 #batch size of 32 performs the best.
epochs = 100
optims= keras.optims.SGD(lr=0.001, momentum=0.9)

model = build_network(optims) 

train_datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.15,
    zoom_range=0.15,
    horizontal_flip=True,  # set each sample mean to 0
    zca_whitening=False,  # apply ZCA whitening
   
)

# Plot the model architecture
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
train_datagen.fit(x_train)

history = model.fit(
        train_datagen.flow(x_train, y_train, batch_size=batch_size),
        validation_data=(x_valid, y_valid),
        steps_per_epoch=len(x_train) / batch_size,
        epochs=epochs,
        callbacks=callbacks,)

In [ ]:
#Builiding RESNET50

IMG_WIDTH=48
IMG_HEIGHT=48
CHANNELS=3
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_WIDTH,IMG_HEIGHT , CHANNELS))

for layer in base_model.layers:
    layer.trainable = True

base_model.summary()
model = models.Sequential()
model.add(base_model)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(7, activation='softmax'))  # 7 classes for facial expressions

# Compile the model
model.compile(loss='categorical_crossentropy',
        optimizer=optims[1],
        metrics=['accuracy'])

# Save the best model during training
checkpoint = ModelCheckpoint('emotion_resnet_model.h5', save_best_only=True)

# Train the model
history = model.fit(
        x_train, y_train, batch_size=batch_size,
        validation_data=(x_valid, y_valid),
        steps_per_epoch=len(x_train) / batch_size,
        epochs=epochs,
        callbacks=[checkpoint]
)


**Displaying the results**

In [ ]:
sns.set()
fig = pyplot.figure(0, (12, 4))

ax = pyplot.subplot(1, 2, 1)
sns.lineplot(x=history.epoch,y=history.history['accuracy'], label='train')
sns.lineplot(x=history.epoch,y=history.history['val_accuracy'], label='valid')
pyplot.title('Accuracy')
pyplot.tight_layout()

ax = pyplot.subplot(1, 2, 2)
sns.lineplot(x=history.epoch,y= history.history['loss'], label='train')
sns.lineplot(x=history.epoch,y= history.history['val_loss'], label='valid')
pyplot.title('Loss')
pyplot.tight_layout()
pyplot.savefig('epoch_history_dnn.png')
pyplot.show()

In [ ]:
print(f'total wrong validation predictions: {np.sum(y_valid) != predicted_classes}\n\n')
print(classification_report(true_classes,predicted_classes))

In [ ]:
train_evalation = model.evaluate(x_train, y_train)
test_evaluation = model.evaluate(x_valid, y_valid)

In [ ]:
metric_list = ["Loss","Accuracy", "Precision", "Recall", "Auc", "F1_Score"]

df = pd.DataFrame(list(zip(metric_list, 
                           [round(num, 3) for num in train_evalation], 
                           [round(num, 3) for num in test_evaluation])),
                  columns=['Metric Name', 'Train', 'Validate'])

print('\n\nModel Scores\n')

HTML(df.to_html(escape=False))

In [ ]:

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()

y_test_bin = label_binarize(y_valid, classes=np.arange(7))
#X_train, X_valid, y_train, y_valid
y_score = model.predict(x_valid)

for i in range(7):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot the ROC curve for each class
plt.figure(figsize=(8, 6))

for i in range(7):
    plt.plot(fpr[i], tpr[i], label=f'Class {i} (AUC = {roc_auc[i]:.2f})')

plt.plot([0, 1], [0, 1], color='navy', linestyle='--', linewidth=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for Multi-class Classification')
plt.legend(loc="lower right")
plt.show()
plt.savefig('roc_dnn')